<a href="https://colab.research.google.com/github/namra98/Deep-Learning/blob/master/Machine_Learning_Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import multivariate_normal
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [166]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !wget https://pjreddie.com/media/files/mnist_train.csv
# !wget https://pjreddie.com/media/files/mnist_test.csv

In [0]:
# dfX = pd.read_csv('/kaggle/input/mnist-original-dataset/train_images_mnist.csv')
# dfY = pd.read_csv('/kaggle/input/mnist-original-dataset/train_labels_mnist.csv')

train = pd.read_csv('/content/drive/My Drive/datasets/mnist_train.csv', header=None)
test = pd.read_csv('/content/drive/My Drive/datasets/mnist_test.csv', header=None)


In [0]:
Y_train = train[0].copy()
X_train = train.copy()
X_train = X_train.drop([0], axis=1)

Y_test = test[0].copy()
X_test = test.copy()
X_test = X_test.drop([0], axis=1).copy()


#LDA Using Two Class (>=5, <5)

In [171]:
X_train_bin = X_train.copy()
Y_train_bin = Y_train.copy()
Y_train_bin[Y_train_bin < 5] = 0
Y_train_bin[Y_train_bin >= 5] = 1

X_test_bin = X_test.copy()
Y_test_bin = Y_test.copy()
Y_test_bin[Y_test_bin < 5] = 0
Y_test_bin[Y_test_bin >= 5] = 1


print(accuracy_score(LDA(X_train_bin.values,Y_train_bin.values,X_test_bin.values),Y_test_bin.values))

(10000, 2)
0.7554


## LDA Function

In [0]:
def predict(X,mean_vec,prior):
    preds = np.zeros((X.shape[0],prior.shape[0]))
    for k in range(prior.shape[0]):
        for i in range(X.shape[0]):
            preds[i][k]=-0.5*np.matmul((X[i,:,k]-mean_vec[k,:]),(X[i,:,k]-mean_vec[k,:]).T)+np.log(prior[k]);
    print(preds.shape);
    return np.argmax(preds,axis=1);

In [0]:
def LDA(X,Y,X_test):
    classes = np.unique(Y);
    mean_vec = np.zeros((classes.shape[0],X.shape[1]))
    prior = np.zeros(classes.shape[0])
    cov_mat = np.zeros((X.shape[1],X.shape[1],classes.shape[0]))
    X_testf = np.zeros((X_test.shape[0],X_test.shape[1],classes.shape[0]))
    for k in range(classes.shape[0]):
        X_k = X[np.where(Y==classes[k])[0]][:];
        mean_vec[k][:] = np.mean(X_k,axis=0);
        prior[k] = float(X_k.shape[0])/X.shape[0];
        X_testf[:,:,k] = X_test;
    preds = predict(X_testf,mean_vec,prior)
    for i in range(X_test.shape[0]):
        preds[i] = classes[preds[i]]
    return preds;

## LDA Using 10 classes

In [174]:
print(accuracy_score(LDA(X_train.values,Y_train.values,X_test.values),Y_test.values))

(10000, 10)
0.8203


In [0]:
def QDA(X,Y,X_test):
    classes = np.unique(Y);
    mean_vec = np.zeros((classes.shape[0],X.shape[1]))
    prior = np.zeros(classes.shape[0])
    cov_mat = np.zeros((X.shape[1],X.shape[1],classes.shape[0]))
    funcs = []
    for k in range(classes.shape[0]):
        X_k = X[np.where(Y==k)[0]][:];
        mean_vec[k][:] = np.mean(X_k,axis=0);
        prior[k] = float(X_k.shape[0])/X.shape[0];
        cov_mat[:,:,k] = np.cov(X_k,rowvar = False);
        funcs.append(multivariate_normal(mean = mean_vec[k][:],cov=cov_mat[:,:,k],allow_singular = True))
    preds = np.zeros(X_test.shape[0])
    for i in range(X_test.shape[0]):
        tmp_preds = np.zeros(classes.shape[0])
        for k in range (classes.shape[0]):
            tmp_preds[k]=funcs[k].pdf(X_test[i][:])*prior[k]
        preds[i] = np.argmax(tmp_preds,axis=0)
    return preds.astype(np.int32);

In [0]:
X_train = pd.read_csv('/content/drive/My Drive/datasets/train_images_mnist.csv', header=None).values
X_test = pd.read_csv('/content/drive/My Drive/datasets/test_images_mnist.csv', header=None).values
Y_train = pd.read_csv('/content/drive/My Drive/datasets/train_labels_mnist.csv', header=None).values
Y_test = pd.read_csv('/content/drive/My Drive/datasets/test_labels_mnist.csv', header=None).values

In [177]:
X_train.shape

(60000, 784)

## QDA Using 10 classes

In [178]:
print(accuracy_score(QDA(X_train,Y_train,X_test),Y_test))

/usr/local/lib/python3.6/dist-packages/scipy/stats/_multivariate.py:750: RuntimeWarning: overflow encountered in exp
  return np.exp(self.logpdf(x))


0.6935


In [179]:
digits = load_digits()
X = digits.data
Y = digits.target
X_tr,X_te,Y_tr,Y_te = train_test_split(X, Y, test_size=0.1, random_state=42)

print(accuracy_score(QDA(X_tr,Y_tr,X_te),Y_te))


0.9555555555555556
